# Relax Challenge Notebook

Objective: determine what factors will predict future user adoption

Adoption is defined as: logging onto the system 3 times in 7 days

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os
import datetime

In [2]:
# Set Directory locations
main_dir = os.path.dirname(os.path.realpath(__name__))
print(main_dir)

/Users/steveyyp/Desktop/relax_challenge


In [3]:
# Load Data
path = os.path.join(main_dir, 'takehome_user_engagement.csv')
df_engagement = pd.read_csv(path)

path = os.path.join(main_dir, 'takehome_users.csv')
df_users = pd.read_csv(path, encoding='latin-1')

# Data Analysis/Clean Data

Examine Data Frames and look at any missing or spurious data

In [4]:
df_engagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [5]:
df_engagement.tail()

,time_stamp,user_id,visited
207912,2013-09-06 06:14:15,11996,1
207913,2013-01-15 18:28:37,11997,1
207914,2014-04-27 12:45:16,11998,1
207915,2012-06-02 11:55:59,11999,1
207916,2014-01-26 08:57:12,12000,1


In [6]:
df_engagement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   time_stamp  207917 non-null  object
 1   user_id     207917 non-null  int64 
 2   visited     207917 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [7]:
df_engagement.describe()

,user_id,visited
count,207917.000000,207917.0
mean,5913.314197,1.0
std,3394.941674,0.0
min,1.000000,1.0
25%,3087.000000,1.0
50%,5682.000000,1.0
75%,8944.000000,1.0
max,12000.000000,1.0


In [8]:
df_engagement['time_stamp'].min(), df_engagement['time_stamp'].max()

('2012-05-31 08:20:06', '2014-06-06 14:58:50')

In [9]:
# First Time Stamp
start_date = df_engagement.sort_values(by='time_stamp').iloc[0]
start_date

time_stamp    2012-05-31 08:20:06
user_id                     10012
visited                         1
Name: 178140, dtype: object

In [10]:
# Last Time Stamp
end_date = df_engagement.sort_values(by='time_stamp', ascending=False).iloc[0]
end_date

time_stamp    2014-06-06 14:58:50
user_id                      4051
visited                         1
Name: 70763, dtype: object

In [11]:
df_users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [12]:
# We can see that there are null values in the 'last-session-creation-time' column and invited_by_user_id column
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   object_id                   12000 non-null  int64  
 1   creation_time               12000 non-null  object 
 2   name                        12000 non-null  object 
 3   email                       12000 non-null  object 
 4   creation_source             12000 non-null  object 
 5   last_session_creation_time  8823 non-null   float64
 6   opted_in_to_mailing_list    12000 non-null  int64  
 7   enabled_for_marketing_drip  12000 non-null  int64  
 8   org_id                      12000 non-null  int64  
 9   invited_by_user_id          6417 non-null   float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


In [13]:
df_users.describe()

,object_id,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
count,12000.00000,8.823000e+03,12000.000000,12000.000000,12000.000000,6417.000000
mean,6000.50000,1.379279e+09,0.249500,0.149333,141.884583,5962.957145
std,3464.24595,1.953116e+07,0.432742,0.356432,124.056723,3383.761968
min,1.00000,1.338452e+09,0.000000,0.000000,0.000000,3.000000
25%,3000.75000,1.363195e+09,0.000000,0.000000,29.000000,3058.000000
50%,6000.50000,1.382888e+09,0.000000,0.000000,108.000000,5954.000000
75%,9000.25000,1.398443e+09,0.000000,0.000000,238.250000,8817.000000
max,12000.00000,1.402067e+09,1.000000,1.000000,416.000000,11999.000000


In [14]:
# Examine Null values in last_session_creation_time
df_users[df_users['last_session_creation_time'].isnull()]

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
7,8,2013-07-31 05:34:02,Hamilton Danielle,DanielleHamilton@yahoo.com,PERSONAL_PROJECTS,NaN,1,1,74,NaN
8,9,2013-11-05 04:04:24,Amsel Paul,PaulAmsel@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,302,NaN
11,12,2014-04-17 23:48:38,Mathiesen Lærke,LaerkeLMathiesen@cuvox.de,ORG_INVITE,NaN,0,0,130,9270.0
14,15,2013-07-16 21:33:54,Theiss Ralf,RalfTheiss@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,175,NaN
15,16,2013-02-11 10:09:50,Engel René,ReneEngel@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,211,NaN
...,...,...,...,...,...,...,...,...,...,...
11975,11976,2013-12-25 22:01:41,Kohl Leah,LeahKohl@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,248,NaN
11977,11978,2014-04-23 16:28:06,Castro Pedro,PedroCunhaCastro@gustr.com,PERSONAL_PROJECTS,NaN,1,0,29,NaN
11984,11985,2013-07-08 17:23:26,Jespersen Marcus,MarcusTJespersen@cuvox.de,PERSONAL_PROJECTS,NaN,0,0,74,NaN
11992,11993,2013-03-28 23:24:21,Townsend Isabel,IsabelTownsend@cuvox.de,PERSONAL_PROJECTS,NaN,0,0,281,NaN


In [15]:
df_users['last_session_creation_time'].fillna(0, inplace=True)

In [16]:
df_users[~df_users['last_session_creation_time'].isnull()]

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0
...,...,...,...,...,...,...,...,...,...,...
11995,11996,2013-09-06 06:14:15,Meier Sophia,SophiaMeier@gustr.com,ORG_INVITE,1.378448e+09,0,0,89,8263.0
11996,11997,2013-01-10 18:28:37,Fisher Amelie,AmelieFisher@gmail.com,SIGNUP_GOOGLE_AUTH,1.358275e+09,0,0,200,NaN
11997,11998,2014-04-27 12:45:16,Haynes Jake,JakeHaynes@cuvox.de,GUEST_INVITE,1.398603e+09,1,1,83,8074.0
11998,11999,2012-05-31 11:55:59,Faber Annett,mhaerzxp@iuxiw.com,PERSONAL_PROJECTS,1.338638e+09,0,0,6,NaN


In [17]:
# Examine Null values in invited_by_user_id
df_users[df_users['invited_by_user_id'].isnull()]

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
6,7,2012-12-16 13:24:32,Sewell Tyler,TylerSewell@jourrapide.com,SIGNUP,1.356010e+09,0,1,37,NaN
7,8,2013-07-31 05:34:02,Hamilton Danielle,DanielleHamilton@yahoo.com,PERSONAL_PROJECTS,0.000000e+00,1,1,74,NaN
8,9,2013-11-05 04:04:24,Amsel Paul,PaulAmsel@hotmail.com,PERSONAL_PROJECTS,0.000000e+00,0,0,302,NaN
10,11,2013-12-26 03:55:54,Paulsen Malthe,MaltheAPaulsen@gustr.com,SIGNUP,1.388117e+09,0,0,69,NaN
13,14,2012-10-11 16:14:33,Rivera Bret,BretKRivera@gmail.com,SIGNUP,1.350058e+09,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...
11992,11993,2013-03-28 23:24:21,Townsend Isabel,IsabelTownsend@cuvox.de,PERSONAL_PROJECTS,0.000000e+00,0,0,281,NaN
11994,11995,2013-10-21 13:33:17,Werfel Sebastian,SebastianWerfel@gmail.com,PERSONAL_PROJECTS,1.382449e+09,0,0,0,NaN
11996,11997,2013-01-10 18:28:37,Fisher Amelie,AmelieFisher@gmail.com,SIGNUP_GOOGLE_AUTH,1.358275e+09,0,0,200,NaN
11998,11999,2012-05-31 11:55:59,Faber Annett,mhaerzxp@iuxiw.com,PERSONAL_PROJECTS,1.338638e+09,0,0,6,NaN


These NaN values can be converted into 0, to denote no user has invited them.

In [18]:
# CONVERT last_session_creation_time to datetime

In [19]:
last_session_creation_time = df_users['last_session_creation_time']

In [20]:
last_session_creation_time.apply(lambda x: datetime.datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S'))

0        2014-04-21 23:53:30
1        2014-03-30 23:45:04
2        2013-03-19 19:14:52
3        2013-05-22 04:09:28
4        2013-01-22 05:14:20
                ...         
11995    2013-09-06 02:14:15
11996    2013-01-15 13:28:37
11997    2014-04-27 08:45:16
11998    2012-06-02 07:55:59
11999    2014-01-26 03:57:12
Name: last_session_creation_time, Length: 12000, dtype: object

In [21]:
df_users['last_session_creation_time'] = last_session_creation_time.apply(lambda x: datetime.datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S'))

In [22]:
df_users['invited_by_user_id'].fillna(0, inplace=True)

In [23]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   object_id                   12000 non-null  int64  
 1   creation_time               12000 non-null  object 
 2   name                        12000 non-null  object 
 3   email                       12000 non-null  object 
 4   creation_source             12000 non-null  object 
 5   last_session_creation_time  12000 non-null  object 
 6   opted_in_to_mailing_list    12000 non-null  int64  
 7   enabled_for_marketing_drip  12000 non-null  int64  
 8   org_id                      12000 non-null  int64  
 9   invited_by_user_id          12000 non-null  float64
dtypes: float64(1), int64(4), object(5)
memory usage: 937.6+ KB


In [24]:
df_users

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,2014-04-21 23:53:30,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-30 23:45:04,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,2013-03-19 19:14:52,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,2013-05-22 04:09:28,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,2013-01-22 05:14:20,0,0,193,5240.0
...,...,...,...,...,...,...,...,...,...,...
11995,11996,2013-09-06 06:14:15,Meier Sophia,SophiaMeier@gustr.com,ORG_INVITE,2013-09-06 02:14:15,0,0,89,8263.0
11996,11997,2013-01-10 18:28:37,Fisher Amelie,AmelieFisher@gmail.com,SIGNUP_GOOGLE_AUTH,2013-01-15 13:28:37,0,0,200,0.0
11997,11998,2014-04-27 12:45:16,Haynes Jake,JakeHaynes@cuvox.de,GUEST_INVITE,2014-04-27 08:45:16,1,1,83,8074.0
11998,11999,2012-05-31 11:55:59,Faber Annett,mhaerzxp@iuxiw.com,PERSONAL_PROJECTS,2012-06-02 07:55:59,0,0,6,0.0


After examination, we found out that the df_engagement dataframe had no null or mission values and was ready for transformations. However, the df_users dataframe contained null and data that was not yet transformation ready. We ended up converting null values from the 'invited_by_user_id' and 'last_session_creation_time' to 0. Afterwhich the unix timestamp values from 'last_session_creation_time' was converted into datetime values

# Data Preparation

This section of the notebook will prepare the data for examination of meaningful features.

In [25]:
# Count Number of times each user visited
df_engagement[['user_id', 'visited']].groupby('user_id').count()

,visited
user_id,
1,1
2,14
3,1
4,1
5,1
...,...
11996,1
11997,1
11998,1


In [26]:
# Count number of users with more than 3 visits
df_counts = df_engagement[['user_id', 'visited']].groupby('user_id').count()
df_counts[df_counts['visited'] > 3]

,visited
user_id,
2,14
10,284
20,7
33,18
42,342
...,...
11975,216
11980,8
11981,4


Creation of a new dataframe that has the date as the index, number of logins over the past 7 days and user id

In [27]:
df_engagement = df_engagement.set_index(pd.DatetimeIndex(df_engagement['time_stamp']))

In [28]:
df_engagement_weekly = df_engagement.groupby('user_id').resample('1W').sum()

In [29]:
df_engagement_weekly = df_engagement_weekly[df_engagement_weekly['user_id'].isnull()==False].drop('user_id',axis=1)

In [30]:
df_engagement_weekly

visited
user_id time_stamp         
1       2014-04-27        1
2       2013-11-17        1
        2013-11-24        0
        2013-12-01        1
        2013-12-08        0
...                     ...
11996   2013-09-08        1
11997   2013-01-20        1
11998   2014-04-27        1
11999   2012-06-03        1
12000   2014-01-26        1

[82282 rows x 1 columns]

In [31]:
active_users = df_engagement_weekly['visited']>=3

In [32]:
active_users = active_users.groupby(level=0).apply(np.sum)

In [33]:
active_users = active_users[active_users!=0]
active_users = pd.DataFrame(active_users,index=active_users.index)
active_users.columns = ['active_weeks']

In [34]:
active_users

,active_weeks
user_id,
2,1
10,52
20,1
33,1
42,68
...,...
11965,1
11967,8
11969,22


Now that we have the number of weeks an active-user was active we can merge it with the df_users dataframe to gain insight on what makes users active

In [35]:
df_complete = active_users.merge(df_users,left_index=True,right_on='object_id',how='inner')

In [36]:
df_complete.head()

,active_weeks,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
1,1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-30 23:45:04,0,0,1,316.0
9,52,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,ORG_INVITE,2014-06-03 18:08:03,1,1,318,4143.0
19,1,20,2014-03-06 11:46:38,Helms Mikayla,lqyvjilf@uhzdq.com,SIGNUP,2014-05-29 07:46:38,0,0,58,0.0
32,1,33,2014-03-11 06:29:09,Araujo José,JoseMartinsAraujo@cuvox.de,GUEST_INVITE,2014-05-31 02:29:09,0,0,401,79.0
41,68,42,2012-11-11 19:05:07,Pinto Giovanna,GiovannaCunhaPinto@cuvox.de,SIGNUP,2014-05-25 15:05:07,1,0,235,0.0


In [37]:
# Add new column of 'active' users and set to 0 == default value
df_users['active'] = int(0)

# If user_id is in df_complete, set active to 1
df_users.iloc[list(df_complete.index),10] = 1

# Sort data by object_id (data used to determine prediction factors)
prediction_data = df_users.set_index('object_id').sort_index()

Data integrity check

In [38]:
prediction_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 1 to 12000
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   creation_time               12000 non-null  object 
 1   name                        12000 non-null  object 
 2   email                       12000 non-null  object 
 3   creation_source             12000 non-null  object 
 4   last_session_creation_time  12000 non-null  object 
 5   opted_in_to_mailing_list    12000 non-null  int64  
 6   enabled_for_marketing_drip  12000 non-null  int64  
 7   org_id                      12000 non-null  int64  
 8   invited_by_user_id          12000 non-null  float64
 9   active                      12000 non-null  int64  
dtypes: float64(1), int64(4), object(5)
memory usage: 1.0+ MB


In [39]:
prediction_data.describe()

,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,active
count,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000
mean,0.249500,0.149333,141.884583,3188.691333,0.120417
std,0.432742,0.356432,124.056723,3869.027693,0.325462
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,29.000000,0.000000,0.000000
50%,0.000000,0.000000,108.000000,875.000000,0.000000
75%,0.000000,0.000000,238.250000,6317.000000,0.000000
max,1.000000,1.000000,416.000000,11999.000000,1.000000


In [40]:
prediction_data.head(10)

,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,active
object_id,,,,,,,,,,
1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,2014-04-21 23:53:30,1,0,11,10803.0,0
2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-30 23:45:04,0,0,1,316.0,1
3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,2013-03-19 19:14:52,0,0,94,1525.0,0
4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,2013-05-22 04:09:28,0,0,1,5151.0,0
5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,2013-01-22 05:14:20,0,0,193,5240.0,0
6,2013-12-17 03:37:06,Cunha Eduardo,EduardoPereiraCunha@yahoo.com,GUEST_INVITE,2013-12-18 22:37:06,0,0,197,11241.0,0
7,2012-12-16 13:24:32,Sewell Tyler,TylerSewell@jourrapide.com,SIGNUP,2012-12-20 08:24:32,0,1,37,0.0,0
8,2013-07-31 05:34:02,Hamilton Danielle,DanielleHamilton@yahoo.com,PERSONAL_PROJECTS,1969-12-31 19:00:00,1,1,74,0.0,0
9,2013-11-05 04:04:24,Amsel Paul,PaulAmsel@hotmail.com,PERSONAL_PROJECTS,1969-12-31 19:00:00,0,0,302,0.0,0


In [41]:
# Drop Unnecessary Columns
prediction_data = prediction_data[['creation_source', 'opted_in_to_mailing_list', 'enabled_for_marketing_drip', 'org_id', 'invited_by_user_id', 'active']]

In [42]:
prediction_data

,creation_source,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,active
object_id,,,,,,
1,GUEST_INVITE,1,0,11,10803.0,0
2,ORG_INVITE,0,0,1,316.0,1
3,ORG_INVITE,0,0,94,1525.0,0
4,GUEST_INVITE,0,0,1,5151.0,0
5,GUEST_INVITE,0,0,193,5240.0,0
...,...,...,...,...,...,...
11996,ORG_INVITE,0,0,89,8263.0,0
11997,SIGNUP_GOOGLE_AUTH,0,0,200,0.0,0
11998,GUEST_INVITE,1,1,83,8074.0,0


In [43]:
# Convert 'creation_source' to one-hot labels
prediction_data = pd.get_dummies(data = prediction_data, columns = ['creation_source'])
prediction_data

,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,active,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH
object_id,,,,,,,,,,
1,1,0,11,10803.0,0,1,0,0,0,0
2,0,0,1,316.0,1,0,1,0,0,0
3,0,0,94,1525.0,0,0,1,0,0,0
4,0,0,1,5151.0,0,1,0,0,0,0
5,0,0,193,5240.0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
11996,0,0,89,8263.0,0,0,1,0,0,0
11997,0,0,200,0.0,0,0,0,0,0,1
11998,1,1,83,8074.0,0,1,0,0,0,0


In [48]:
# Convert org_id to one-hot labels
org = prediction_data.groupby('org_id').sum()

org_ids = []
for i in prediction_data['org_id']:
    ids = org.loc[i,'active']
    org_ids.append(ids)
prediction_data['org_id'] = org_ids

prediction_data = pd.get_dummies(data=prediction_data, columns=['org_id'])
prediction_data

,opted_in_to_mailing_list,enabled_for_marketing_drip,invited_by_user_id,active,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH,org_id_0,...,org_id_6,org_id_7,org_id_8,org_id_9,org_id_10,org_id_11,org_id_12,org_id_13,org_id_14,org_id_16
object_id,,,,,,,,,,,,,,,,,,,,,
1,1,0,10803.0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,316.0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,1525.0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,5151.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,0,0,5240.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
prediction_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 1 to 12000
Data columns (total 25 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   opted_in_to_mailing_list            12000 non-null  int64  
 1   enabled_for_marketing_drip          12000 non-null  int64  
 2   invited_by_user_id                  12000 non-null  float64
 3   active                              12000 non-null  int64  
 4   creation_source_GUEST_INVITE        12000 non-null  uint8  
 5   creation_source_ORG_INVITE          12000 non-null  uint8  
 6   creation_source_PERSONAL_PROJECTS   12000 non-null  uint8  
 7   creation_source_SIGNUP              12000 non-null  uint8  
 8   creation_source_SIGNUP_GOOGLE_AUTH  12000 non-null  uint8  
 9   org_id_0                            12000 non-null  uint8  
 10  org_id_1                            12000 non-null  uint8  
 11  org_id_2                            12000

# Feature Importance

This section of the notebook will create a model and identify which features are the most important at determining if a user will be active

In [50]:
# Partition Data into features and labels
y = prediction_data.pop('active')
X = prediction_data

In [60]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV

In [70]:
params = {'max_depth':[3, 5, 6, 8, 10, 15, 25], 
          'eta':[0.01, 0.05, 0.1, 0.3, 0.5], 
          'gamma': [0, 1, 2, 3, 4],
          'n_estimators': [40, 50, 60, 70, 80]}

In [71]:
# Create Training Matrix
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=23)
dtrain = xgb.DMatrix(data = X_train, label = y_train)
dtest = xgb.DMatrix(data = X_test, label = y_test)

In [80]:
# Define Model
xgb_model = xgb.XGBClassifier()

In [81]:
# Create GridSearch
clf = GridSearchCV(xgb_model, 
                   params, 
                   n_jobs = -1,
                   cv = 5,
                   scoring='accuracy')

In [82]:
# Train Model
clf.fit(X_train, y_train)

/Users/steveyyp/anaconda3/envs/tf_2/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constrai...
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_method=None,
                                     validate_parameters=False,
                                     

In [83]:
# Get Best Values
best_para_xgb = clf.best_params_
best_acc_xgb = clf.best_score_

print(f'Best Parameters:, {best_para_xgb}\n')
print(f'Best Accuracy:, {best_acc_xgb}\n')

Best Parameters:, {'eta': 0.1, 'gamma': 1, 'max_depth': 6, 'n_estimators': 80}

Best Accuracy:, 0.8795555555555555



In [84]:
# Retrain model on best parameters
xgb_model_perf = xgb.XGBClassifier(n_estimators = best_para_xgb['n_estimators'],
                                   eta = best_para_xgb['eta'],
                                   gamma = best_para_xgb['gamma'], 
                                   max_depth = best_para_xgb['max_depth'])
xgb_model_perf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.1, gamma=1,
              gpu_id=-1, importance_type='gain', interaction_constraints=None,
              learning_rate=0.100000001, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=80, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [85]:
# Get Most Important Factors
imp_features = pd.DataFrame(xgb_model_perf.feature_importances_).transpose()
imp_features.columns = list(prediction_data.columns)
imp_features.index = ['XGM'] 
imp_features.transpose().sort_values(by='XGM',ascending=False)

,XGM
creation_source_PERSONAL_PROJECTS,0.086881
org_id_8,0.086269
org_id_0,0.074118
org_id_1,0.069017
org_id_2,0.062040
org_id_5,0.055580
org_id_6,0.055133
org_id_11,0.050216
org_id_4,0.047132
org_id_12,0.043536


We can see that the most important feature is the creation_source (Personal Project). This supports a narrative that users that login to create personal projects are the most likely to be active users. The least powerful feature is org_id_16. 